In [1]:
import random
import re
import math

import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as gbm
import gensim

from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, classification_report
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


try to resample it, but it doesn't works.

In [2]:
DayOfWeek = (
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday'
)
Add_dict = {}
Category_encoder = preprocessing.LabelEncoder()

# def resample(data):
#         w = {}
#         # d =  pd.read_csv('./data/weight.csv')
#         remove_list = []
#         add_items = []
#         for index, row in pd.read_csv('./data/weight.csv').iterrows():
#             w[row[0]] = row[1]
#         for index, row in data.iterrows():
#             weight = w[row['Category']]
#             if weight < 1:
#                 if random.random() > weight:
#                     remove_list.append(index)
#             elif weight > 1:
#                 add_items += [row] * int(weight)
#         data.drop(remove_list, inplace=True)
#         res = pd.concat([data, pd.DataFrame(add_items)], ignore_index=True, copy=False)
#         return res
    
data = pd.read_csv('./crime/data/train.csv')
test = pd.read_csv('./crime/data/test.csv')
# data = resample(data)
y=data.Category
Category_encoder.fit(y)
data.drop(['Category', 'Descript', 'Resolution'], axis=1, inplace=True)
test_id = pd.DataFrame({"Id":test.Id})
data.head(5)

,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541


## 时间特征
这里看了一下各个时间点的犯罪统计。
* 年：年份对犯罪数量没什么影响，除了2015年只有一半的时间导致犯罪数量锐减
* 月：基本没有影响，大致稳定在65000~80000范围内
* 日：除了1号稍多，其它日期比较平稳
* 时：这个呈现出比较明显的特征，上午8时开始逐渐上升，两次顶峰出现在12点和18点，凌晨1点开始回落。
* 分：这个特征异常明显，在整点时（分=0）的犯罪数量约占30%，半点时（分=30）的犯罪数量约占14%，在分=1和其它分钟数为5的倍数时，犯罪数量在15000~40000以内波动，远高于其它时间（4000~5000内波动），其中以15、45最高

## 时间处理
* 时：我们认为不同小时的犯罪类型应当是适合在一天的不同时段进行的犯罪，因此按时段分为有序的4类：1-7，8-12，13-18，19-24。但是事实说明这样处理效果不好，所以没有采用。
* 分：我们认为分钟意味着是否有预谋，在整点进行的犯罪通常是有预谋的（分=1可能是因为时间误差或者动手延迟），非整点常常是即兴犯罪，因此分2类：整点（分=0，1，15，30，45），非整点（分=其它）

In [3]:
# process date features
min_feature_set = set((0, 1, 15, 30, 45))

def day_transform(str):
        return DayOfWeek.index(str)

def process_date(x):
        def f(str):
            # year, month, date, hour, minute
            date_list = list(map(lambda x: int(x), re.split('[ :-]', str)[0:5]))
#             hour = date_list[3]
#             if hour >= 1 and hour < 8:
#                 date_list.append(0)
#             elif hour >=8 and hour < 13:
#                 date_list.append(1)
#             elif hour >=13 and hour < 19:
#                 date_list.append(2)
#             else:
#                 date_list.append(3)
            date_list.append(0 if date_list[4] in min_feature_set else 1)
            return date_list

        l = []
        for s in x.Dates:
            l.append(f(s))
        date_frame = pd.DataFrame(np.array(l),
                                  columns=['year', 'month', 'date', 'hour', 'minute', 'minute_feature'])
        date_frame.insert(5, 'DayOfWeek', x.DayOfWeek.apply(day_transform))
        return date_frame
#         x.drop(['Dates'], axis=1, inplace=True)
#         return pd.concat([date_frame, x], axis=1, copy=False)

In [4]:
data_date = process_date(data)
test_date = process_date(test)
data_date.head(5)

,year,month,date,hour,minute,DayOfWeek,minute_feature
0,2015,5,13,23,53,2,1
1,2015,5,13,23,53,2,1
2,2015,5,13,23,33,2,1
3,2015,5,13,23,30,2,0
4,2015,5,13,23,30,2,0


In [5]:
# data.insert(len(data.columns), ['pd' + str(i) for i in range(1, 11)], [pd.get_dummies(data.PdDistrict)])
data_dist = pd.get_dummies(data.PdDistrict)
test_dist = pd.get_dummies(test.PdDistrict)

In [6]:
def process_address(x, clean=True):
    """
    这个函数最终没有使用。
    
    将数据集中的地址进行one-hot编码，对于出现次数小于100的地址，全部归入other。
    若clean为True，表示用于训练集，会统计出现的地址，并计算编码
    若clean为False，表示用于测试集，仅用之前统计的字典将地址编码，
    未出现的地址认为是小概率地址，归入other
    :param x: DataFrame
    :param clean: 是否清除之前统计的数据
    :return: 编码后的DataFrame
    """
    global Add_dict

    def pre(str):
        global Add_dict
        if str in Add_dict:
            Add_dict[str] += 1
        else:
            Add_dict[str] = 0
    
    def trans2(str):
        return str if str in address_set else 'others'

    if clean:
        Add_dict = {}
        x.Address.apply(pre)
        remove_list = []
        for k, v in Add_dict.items():
            if v < 100:
                remove_list.append(k)
        for k in remove_list:
            Add_dict.pop(k)
    address_set = tuple(Add_dict.keys())
    x.Address = x.Address.apply(trans2)
    return pd.get_dummies(x.Address)

In [7]:
"""
将地址进行分词并训练成100维词向量
'/'也作为一个单词进行训练
"""
word2vec_n = 100
data_sentences = []
for s in data.Address:
    data_sentences.append(s.split(' '))
# test_sentences = []
for s in test.Address:
    data_sentences.append(s.split(' '))
data_model = gensim.models.Word2Vec(data_sentences, min_count=1, size=word2vec_n, workers=30)
data_model.train(data_sentences, total_examples=len(data_sentences), epochs=200)

720650570

In [8]:
"""
将地址转换为词向量
"""
def address_transform(str):
    vec=0
    for s in str.split(' '):
        vec+=data_model[s]
    return vec
data_addr_list = []
test_addr_list = []
for add in data.Address:
    data_addr_list.append(address_transform(add))
data_addr = pd.DataFrame(data_addr_list, columns=['addr_%d' % i for i in range(0, word2vec_n)])
for add in test.Address:
    test_addr_list.append(address_transform(add))
test_addr = pd.DataFrame(test_addr_list, columns=['addr_%d' % i for i in range(0, word2vec_n)])

In [9]:
def process_addr_gmm(n=50):
    """处理Address词向量信息
    使用高斯混合模型对地址的词向量进行聚类
    
    Args: 
        n: 聚类的类别数量
    """
    addr_gmm = mixture.GaussianMixture(n_components=50, covariance_type='diag').fit(data_addr)
    return pd.get_dummies(pd.Series(addr_gmm.predict(data_addr))), pd.get_dummies(pd.Series(addr_gmm.predict(test_addr)))
data_addr_gmm, test_addr_gmm = process_addr_gmm(100)

In [10]:
def process_loc(n):
    """处理经纬度信息
    使用高斯混合模型进行聚类
    
    Args: 
        n: 聚类的类别数量
    """
    gmm = mixture.GaussianMixture(n_components=n, covariance_type='diag').fit(data[['X', 'Y']])
    return pd.get_dummies(pd.Series(gmm.predict(data[['X', 'Y']]))), pd.get_dummies(pd.Series(gmm.predict(test[['X', 'Y']])))
data_loc, test_loc = process_loc(140)
data_loc.head(5)

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
 pd.concat([data_date, data_loc, data_dist, data_addr], axis=1, copy=False).head(5)

,year,month,date,hour,minute,DayOfWeek,minute_feature,0,1,2,...,addr_90,addr_91,addr_92,addr_93,addr_94,addr_95,addr_96,addr_97,addr_98,addr_99
0,2015,5,13,23,53,2,1,0,0,0,...,-3.546991,-2.134894,-1.915281,2.306646,1.333409,-2.913253,2.142856,-0.980920,-3.165184,5.596323
1,2015,5,13,23,53,2,1,0,0,0,...,-3.546991,-2.134894,-1.915281,2.306646,1.333409,-2.913253,2.142856,-0.980920,-3.165184,5.596323
2,2015,5,13,23,33,2,1,0,0,0,...,0.123078,-0.874606,-1.186695,-2.600078,-1.450801,1.797596,2.760073,1.360794,-2.979972,4.049474
3,2015,5,13,23,30,2,0,0,0,0,...,1.092991,-2.009022,2.491399,-1.745062,0.811430,3.055635,-0.426461,-1.896576,0.339530,-2.595366
4,2015,5,13,23,30,2,0,0,0,0,...,1.056252,-2.127862,2.573607,2.720907,0.431179,0.781310,2.623312,-1.407373,2.989151,-0.091687


- boosting_type (string, optional (default="gbdt")) – ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest.
- num_leaves (int, optional (default=31)) – Maximum tree leaves for base learners.
- max_depth (int, optional (default=-1)) – Maximum tree depth for base learners, -1 means no limit.
- learning_rate (float, optional (default=0.1)) – Boosting learning rate.
- n_estimators (int, optional (default=100)) – Number of boosted trees to fit.
- subsample_for_bin (int, optional (default=50000)) – Number of samples for constructing bins.
- objective (string, callable or None, optional (default=None)) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below). default: ‘regression’ for LGBMRegressor, ‘binary’ or ‘multiclass’ for LGBMClassifier, ‘lambdarank’ for LGBMRanker.
- class_weight (dict, 'balanced' or None, optional (default=None)) – Weights associated with classes in the form {class_label: weight}. Use this parameter only for multi-class classification task; for binary classification task you may use is_unbalance or scale_pos_weight parameters. The ‘balanced’ mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)). If None, all classes are supposed to have weight one. Note that these weights will be multiplied with sample_weight (passed through the fit method) if sample_weight is specified.
- min_split_gain (float, optional (default=0.)) – Minimum loss reduction required to make a further partition on a leaf node of the tree.
- min_child_weight (float, optional (default=1e-3)) – Minimum sum of instance weight(hessian) needed in a child(leaf).
- min_child_samples (int, optional (default=20)) – Minimum number of data need in a child(leaf).
- subsample (float, optional (default=1.)) – Subsample ratio of the training instance.
- subsample_freq (int, optional (default=1)) – Frequence of subsample, <=0 means no enable.
- colsample_bytree (float, optional (default=1.)) – Subsample ratio of columns when constructing each tree.
- reg_alpha (float, optional (default=0.)) – L1 regularization term on weights.
- reg_lambda (float, optional (default=0.)) – L2 regularization term on weights.
- random_state (int or None, optional (default=None)) – Random number seed. Will use default seeds in c++ code if set to None.
- n_jobs (int, optional (default=-1)) – Number of parallel threads.
- silent (bool, optional (default=True)) – Whether to print messages while running boosting.

In [12]:
def train(submit=False):
    """训练函数
    通过设定好的参数进行训练
    
    Args:
        submit: 表示是否使用训练集部数据训练模型。如果False，将训练集数据拆分为训练集（70%）和测试集（30%），
        训练完成后分别测试模型在训练集和测试集上的表现；如果True，将训练集数据用于训练模型，输出模型在训练集上的表现，
        然后使用kaggle测试集进行预测并保存预测结果用于提交。
    """
    param = dict(max_depth=8, 
                 n_estimators=350, 
                 n_jobs=32, 
                 objective='multiclass', 
                 learning_rate=0.05, 
                 subsample=0.6, 
                 num_leaves=128)
    
    if submit:
        X_train = pd.concat([data_date, data_loc, data_dist, data_addr], axis=1, copy=False)
        y_train = Category_encoder.transform(y)
        X_test = pd.concat([test_date, test_loc, test_dist, test_addr], axis=1, copy=False)
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            pd.concat([data_date, data_loc, data_dist, data_addr], axis=1, copy=False), Category_encoder.transform(y), 
            test_size=0.3, random_state=0)

    classifier = gbm.LGBMClassifier(**param)
    classifier.fit(X_train, y_train, eval_metric='mlogloss')

    result_train = pd.DataFrame(classifier.predict_proba(X_train))
    result_test = pd.DataFrame(classifier.predict_proba(X_test))
    
    print('train:', log_loss(y_train, result_train))
    if submit:
        prefix=''
        for k, v in  param.items():
            prefix += '%s=%s.' % (k, str(v))
        filepath = './crime/%scsv' % prefix
        result = pd.DataFrame(result_train)
        result_test.insert(0, 'Id', test.Id)
        result_test.columns = result_test.columns.map(lambda m: Category_encoder.inverse_transform(m) if isinstance(m, int) else m)
        result_test.to_csv(filepath, index=False)
        print('Finished saving files. Filename:\n' + filepath)
    else:
        print('test:', log_loss(y_test, result_test), '\n')
    return classifier

train(submit=False)
# classifier = train(submit=True)

train: 1.84409965802
test: 2.22612029068 



LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.05, max_depth=8, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=350,
        n_jobs=32, num_leaves=128, objective='multiclass',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.6, subsample_for_bin=200000, subsample_freq=1)


## xgboost:
------------------------------
### 最开始调的参数，但是并不适用于LightGBM
xgboost.gmm=39.date=hour-minute-featured-with-DayOfWeek.loc=one-hot.dist=yes.addr=no.max_depth=8.n_estimators=100
xgboost.gmm=39.date=DayOfWeek.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50 - train: 2.34525, test:2.42062
xgboost.gmm=39.date=minute-featured-DayOfWeek.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50 - train: 2.3399, test: 2.4182
xgboost.gmm=39.date=hour-featured-DayOfWeek.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50 - train:2.34595, test: 2.42104


xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50 —— test:2.395
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=100 —— train: 2.278, test: 2.40
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50,subsample=0.5 —— train:2.36300, test:2.43554
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50,subsample=0.6 —— train:2.36127, test:2.43443
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50,subsample=0.7 —— train:2.3598, test:2.4346
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50,subsample=0.8 —— train:2.3601, test:2.4345
xgboost.gmm=20.loc=one-hot.dist=one-hot.addr=no.max_depth=8.n_estimators=50,subsample=0.9 —— train:2.3605, test:2.4347

### learning_rate = 0.1

###### n_estimators: 50
- train: 2.3475846850753084
- test: 2.4300819496329393 

###### n_estimators: 100
- train: 2.2493114099258396
- test: 2.3980333707882386 

###### n_estimators: 150
- train: 2.183500019381262
- test: 2.3947450429680432 

###### n_estimators: 200
- train: 2.124935285254291
- test: 2.3954496446712015 

###### n_estimators: 250
- train: 2.073948376885028
- test: 2.398131535806666 

###### max_depth=8, n_estimators=300, gamma=0.1:
- train: 2.0262884178248872
- test: 2.401822482656742 

###### max_depth=6, n_estimators=300, gamma=0.1:
- train: 2.25023725734
- test: 2.39739200022 
(学习能力明显下降，泛化能力明显上升）

###### max_depth=8, n_estimators=300, gamma=0.1, min_child_weight=100:
- train: 2.31327214908
- test: 2.3994496945 
（emmm泛化能力时提升了，但是这学习能力也下降了啊，最后测试集上的效果反而不好= =）

## LGBM:
----------------------------------------
### learning_rate=0.1, num_leaves=31

###### n_estimators=50
- train: 2.422454054660593
- test: 2.4404583378122604

###### n_estimators=100
- train: 2.3692381371383378
- test: 2.4061435863370733

###### n_estimators=150
- train: 2.344883461603974
- test: 2.4011314814512312

###### n_estimators=200
- train: 2.3248051741150637
- test: 2.3993401253740334

###### n_estimators=250
- train: 2.3067374466734663
- test: 2.398465633267535

###### n_estimators=300
- train: 2.290042564420303
- test: 2.397985343171034

###### n_estimators=350
- train: 2.274443265517788
- test: 2.3979380054115764

### n_estimators=300, learning_rate=0.1, num_leaves=31

###### number of components :  20
* train: 2.2896706778032496
*  test: 2.397423002843296

###### number of components :  30
* train: 2.282489219935824
*  test: 2.388900244514006

###### number of components :  40
* train: 2.2792566711136835
*  test: 2.385515245018877

###### number of components :  50
* train: 2.2648347392932076
*  test: 2.371450279541818

###### number of components :  60
* train: 2.2651821271942696
*  test: 2.370866385085879

###### number of components :  70
* train: 2.2653701804662734
*  test: 2.3699399112568265

###### number of components :  80
* train: 2.263639326249622
*  test: 2.368659649500926

###### number of components :  90
* train: 2.256510999107492
*  test: 2.3603235176030766

###### number of components :  100
* train: 2.2609116781735126
*  test: 2.363845173790893

###### number of components :  110
* train: 2.257224577635038
*  test: 2.3595274867252343

###### number of components :  120
* train: 2.2519685240667475
*  test: 2.3538891635210915

### n_estimators=300, learning_rate=0.1, num_leaves=128

###### number of components :  110
- train: 2.147998594707557
-  test: 2.3535705137712695

###### number of components :  120
- train: 2.1507204445765296
-  test: 2.351659633317391

###### number of components :  130
- train: 2.1537820299763353
-  test: 2.350372574069103

###### number of components :  140
- train: 2.1544479960148397
-  test: 2.348725349761922

###### number of components :  150
- train: 2.16507444560338
-  test: 2.35556314010956

###### number of components :  160
- train: 2.167476377354809
-  test: 2.356295454033946

##### submit:
- train: 2.16489928137
- test:2.36164

parameters
- max_depth=8, 
- n_estimators=350, 
- n_jobs=32, 
- objective='multiclass', 
- learning_rate=0.1, 
- subsample=0.6, 
- num_leaves=128

##### submit
- train: 2.24521060034
- test: 2.36079

parameters
- max_depth=8, 
- n_estimators=350, 
- n_jobs=32, 
- objective='multiclass', 
- learning_rate=0.05, 
- subsample=0.6, 
- num_leaves=128

这个参数在训练集上的错误率比上一个明显上升，我都没提交，后来死马当活马医交了一下，居然还降低了。

#### Tuning hyper-parameters for neg_log_loss
- 预设参数：{'n_jobs': 32, 'subsample': 0.6, 'objective': 'multiclass', 'max_depth': 8}

Best parameters set found on development set:

- {'num_leaves': 128, 'learning_rate': 0.05, 'n_estimators': 350}

num_leaves越大，学习能力提升明显，泛化能力下降也明显。
learning_rate降低应该要同时增加n_estimators才行。
先调整learning_rate和n_estimators，然后再调整num_leaves和泛化性能吧。

Grid scores on development set:

- train: -1.999 (+/-0.001), test: -2.428 (+/-0.002) for {'num_leaves': 128, 'learning_rate': 0.1, 'n_estimators': 350}
- train: -1.969 (+/-0.001), test: -2.431 (+/-0.002) for {'num_leaves': 170, 'learning_rate': 0.1, 'n_estimators': 350}
- train: -1.963 (+/-0.001), test: -2.432 (+/-0.002) for {'num_leaves': 200, 'learning_rate': 0.1, 'n_estimators': 350}
- train: -1.959 (+/-0.001), test: -2.434 (+/-0.002) for {'num_leaves': 128, 'learning_rate': 0.1, 'n_estimators': 400}
- train: -1.926 (+/-0.001), test: -2.438 (+/-0.002) for {'num_leaves': 170, 'learning_rate': 0.1, 'n_estimators': 400}
- train: -1.920 (+/-0.001), test: -2.439 (+/-0.002) for {'num_leaves': 200, 'learning_rate': 0.1, 'n_estimators': 400}
- train: -2.161 (+/-0.001), test: -2.406 (+/-0.001) for {'num_leaves': 128, 'learning_rate': 0.05, 'n_estimators': 350}
- train: -2.141 (+/-0.000), test: -2.406 (+/-0.002) for {'num_leaves': 170, 'learning_rate': 0.05, 'n_estimators': 350}
- train: -2.138 (+/-0.001), test: -2.406 (+/-0.002) for {'num_leaves': 200, 'learning_rate': 0.05, 'n_estimators': 350}
- train: -2.134 (+/-0.001), test: -2.407 (+/-0.001) for {'num_leaves': 128, 'learning_rate': 0.05, 'n_estimators': 400}
- train: -2.113 (+/-0.000), test: -2.408 (+/-0.002) for {'num_leaves': 170, 'learning_rate': 0.05, 'n_estimators': 400}
- train: -2.108 (+/-0.001), test: -2.408 (+/-0.002) for {'num_leaves': 200, 'learning_rate': 0.05, 'n_estimators': 400}
- train: -2.301 (+/-0.000), test: -2.413 (+/-0.001) for {'num_leaves': 128, 'learning_rate': 0.02, 'n_estimators': 350}
- train: -2.290 (+/-0.000), test: -2.413 (+/-0.001) for {'num_leaves': 170, 'learning_rate': 0.02, 'n_estimators': 350}
- train: -2.287 (+/-0.000), test: -2.412 (+/-0.001) for {'num_leaves': 200, 'learning_rate': 0.02, 'n_estimators': 350}
- train: -2.283 (+/-0.000), test: -2.408 (+/-0.001) for {'num_leaves': 128, 'learning_rate': 0.02, 'n_estimators': 400}
- train: -2.271 (+/-0.000), test: -2.408 (+/-0.001) for {'num_leaves': 170, 'learning_rate': 0.02, 'n_estimators': 400}
- train: -2.268 (+/-0.000), test: -2.408 (+/-0.001) for {'num_leaves': 200, 'learning_rate': 0.02, 'n_estimators': 400}


#### Tuning hyper-parameters for neg_log_loss
- min_child_samples=100 用来抑制过拟合，不知道效果如何（原值为10）
- 预设参数：{'min_child_samples': 100, 'n_jobs': 32, 'num_leaves': 128, 'subsample': 0.6, 'objective': 'multiclass', 'max_depth': 8}

Best parameters set found on development set:
- {'learning_rate': 0.01, 'n_estimators': 400}

Grid scores on development set:
- train: -2.060 (+/-0.031), test: -7.202 (+/-3.426) for {'learning_rate': 0.1, 'n_estimators': 400}
- train: -2.041 (+/-0.031), test: -7.364 (+/-3.457) for {'learning_rate': 0.1, 'n_estimators': 450}
- train: -2.022 (+/-0.031), test: -7.487 (+/-3.491) for {'learning_rate': 0.1, 'n_estimators': 500}
- train: -2.005 (+/-0.031), test: -7.585 (+/-3.520) for {'learning_rate': 0.1, 'n_estimators': 550}
- train: -1.989 (+/-0.031), test: -7.710 (+/-3.553) for {'learning_rate': 0.1, 'n_estimators': 600}
- train: -1.973 (+/-0.032), test: -7.806 (+/-3.557) for {'learning_rate': 0.1, 'n_estimators': 650}
- train: -2.150 (+/-0.028), test: -6.296 (+/-3.192) for {'learning_rate': 0.05, 'n_estimators': 400}
- train: -2.137 (+/-0.029), test: -6.439 (+/-3.276) for {'learning_rate': 0.05, 'n_estimators': 450}
- train: -2.124 (+/-0.029), test: -6.575 (+/-3.336) for {'learning_rate': 0.05, 'n_estimators': 500}
- train: -2.112 (+/-0.030), test: -6.678 (+/-3.364) for {'learning_rate': 0.05, 'n_estimators': 550}
- train: -2.101 (+/-0.030), test: -6.790 (+/-3.391) for {'learning_rate': 0.05, 'n_estimators': 600}
- train: -2.089 (+/-0.030), test: -6.882 (+/-3.408) for {'learning_rate': 0.05, 'n_estimators': 650}
- train: -2.231 (+/-0.021), test: -5.339 (+/-2.646) for {'learning_rate': 0.02, 'n_estimators': 400}
- train: -2.221 (+/-0.022), test: -5.462 (+/-2.738) for {'learning_rate': 0.02, 'n_estimators': 450}
- train: -2.212 (+/-0.023), test: -5.567 (+/-2.847) for {'learning_rate': 0.02, 'n_estimators': 500}
- train: -2.204 (+/-0.023), test: -5.651 (+/-2.908) for {'learning_rate': 0.02, 'n_estimators': 550}
- train: -2.197 (+/-0.024), test: -5.736 (+/-2.966) for {'learning_rate': 0.02, 'n_estimators': 600}
- train: -2.190 (+/-0.025), test: -5.820 (+/-3.002) for {'learning_rate': 0.02, 'n_estimators': 650}
- train: -2.322 (+/-0.014), test: -4.514 (+/-1.548) for {'learning_rate': 0.01, 'n_estimators': 400}
- train: -2.301 (+/-0.016), test: -4.645 (+/-1.743) for {'learning_rate': 0.01, 'n_estimators': 450}
- train: -2.285 (+/-0.016), test: -4.768 (+/-1.920) for {'learning_rate': 0.01, 'n_estimators': 500}
- train: -2.272 (+/-0.017), test: -4.883 (+/-2.079) for {'learning_rate': 0.01, 'n_estimators': 550}
- train: -2.261 (+/-0.018), test: -4.992 (+/-2.225) for {'learning_rate': 0.01, 'n_estimators': 600}
- train: -2.252 (+/-0.019), test: -5.097 (+/-2.363) for {'learning_rate': 0.01, 'n_estimators': 650}

summary
- min_child_samples严重降低了模型的泛化性能（为什么？）
- 随着n_estimators的提升，泛化性能越来越低，降低learning_rate可以提升泛化性能，但是降低了拟合能力。
- 可能是收到min_child_samples的影响，提高n_estimators并没有效果。
- 建议测试：learning_rate=0.01, n_estimators=[350, 400, 450, 500], num_leaves=[128, 170, 200]
- min_child_samples=[10, 20]吧,玩不起
